In [2]:
import requests
from bs4 import BeautifulSoup, Comment

import numpy as np
import pandas as pd
import re

In [3]:
url = 'http://www.okura-movie.co.jp/meguro_cinema/now_showing.html'

In [4]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
def get_soup():
    '''
    html request to meguro cinema website to get soup
    '''
    url = 'http://www.okura-movie.co.jp/meguro_cinema/now_showing.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

### Movies

One Movie

In [6]:
movie = soup.find_all(id="sakuhin_detail")[0]

In [7]:
engtitle = movie.find('span', class_='text_small').text
engtitle

'GHOSTBUSTERS: AFTERLIFE'

In [8]:
pattern = r"タイトル☆-->.*<!--☆ここまで"

jptitle = re.findall(pattern, str(movie))[0].replace('\u3000', ' ')
jptitle

'タイトル☆-->ゴーストバスターズ アフターライフ<!--☆ここまで'

Every Movie

In [9]:
movies = soup.find_all(id="sakuhin_detail")
movies[0]


<div id="sakuhin_detail">
<div class="sakuhin_title">
<p><span class="jp_small2"><!--☆２本立てタイトル☆→-->〜ＧＷはゴースト退治だ！！〜<!--←☆２本立てタイトル☆--></span></p>
</div>
<div class="mgn10"><!-- space --></div>
<div class="sakuhin_title">
<p>
<span class="jp"><!--☆ここから☆作品タイトル☆-->ゴーストバスターズ　アフターライフ<!--☆ここまで☆作品タイトル☆-->
<!--☆ここから☆原題☆--><span class="text_small">GHOSTBUSTERS: AFTERLIFE</span><!--☆ここまで☆原題☆-->
</span>
<span class="bikou"><!--☆ここから☆視聴制限☆--><img alt="■■■" src="img/PG12.gif"/><!--☆ここまで☆視聴制限☆--></span>
</p>
</div><!--//sakuhin_title area.-->
<div class="mgn00"><!-- space --></div>
<div class="sakuhin_story">
<div id="story_r" style="height: 1200px;">
<p class="catch">
<!--☆↓キャッチ↓☆-->
<!--☆↑キャッチ↑☆-->
</p>
<p class="story_type3" style="height: 500px;"><span></span>
<!--☆↓物語↓☆-->
<a href="https://www.ghostbusters.jp/" target="_blank"><img div="" src="img/event/com919.jpg" style="margin-top : 40px" width="650"/></a>
<!--☆↑物語↑☆-->
<p class="story_type3" style="height: 350px;"><span>【物語】</span>
<!--☆↓物語↓☆--

In [10]:
movie_info = []

pattern = r"タイトル☆-->(.*)<!--☆ここまで"

In [11]:
for movie in movies:
    engtitle = movie.find('span', class_='text_small').text
    jptitle = re.findall(pattern, str(movie))[0].replace('\u3000', ' ')
    movie_dict = {'engtitle':engtitle, 'jptitle':jptitle}
    movie_info.append(movie_dict)

In [12]:
for m in movie_info:
    print(m)

{'engtitle': 'GHOSTBUSTERS: AFTERLIFE', 'jptitle': 'ゴーストバスターズ アフターライフ'}
{'engtitle': 'GHOSTBUSTERS', 'jptitle': 'ゴーストバスターズ'}
{'engtitle': 'SPIDER-MAN: NO WAY HOME', 'jptitle': 'スパイダーマン：ノー・ウェイ・ホーム'}
{'engtitle': 'UNCHARTED', 'jptitle': 'アンチャーテッド'}
{'engtitle': 'SPIDER-MAN: INTO THE SPIDER-VERSE', 'jptitle': 'スパイダーマン：スパイダーバース'}
{'engtitle': '', 'jptitle': 'ハッピーアワー'}
{'engtitle': 'EL PLANETA', 'jptitle': 'エル プラネタ'}
{'engtitle': 'LAS NINAS / SCHOOLGIRLS', 'jptitle': 'スクールガールズ'}
{'engtitle': 'The Blues Brothers', 'jptitle': 'ブルース・ブラザース'}
{'engtitle': '（あるいは、革命がテレビ放映されなかった時）\nSUMMER OF SOUL (OR, WHEN THE REVOLUTION COULD NOT BE TELEVISED)\n', 'jptitle': 'サマー・オブ・ソウル<span class="text_small">（あるいは、革命がテレビ放映されなかった時）'}
{'engtitle': 'BELUSHI', 'jptitle': 'ＢＥＬＵＳＨＩ ベルーシ'}
{'engtitle': '', 'jptitle': '新幹線大爆破'}
{'engtitle': '', 'jptitle': '家族ゲーム'}
{'engtitle': '', 'jptitle': '太陽を盗んだ男'}
{'engtitle': '', 'jptitle': '炎の肖像'}
{'engtitle': '怒火・重案 / RAGING FIRE', 'jptitle': 'レイジング・ファイア'}
{'engtitle': '肥龍過江 / 

### Movies + Dates

In [276]:
soup.find_all(id=['jyouei_kikan','sakuhin_detail'])[3] 

#dates and movies are sequential in the html and not distributed into common parents
#so go through the soup searching for both the dates (j_k) and movie info (s_d)

<div id="jyouei_kikan">
<div id="detail_l">
<p><span class="date">5.7</span><span class="week">sat</span><span class="week">〜</span>
<span class="date">5.13</span><span class="week">fri</span></p>
</div>
<div id="detail_r">
<p><span class="period"> 1週間上映 </span></p>
</div>
<div id="detail_line"><!-- line space --></div>
</div>

In [374]:
dateandmovie = soup.find_all(id=['jyouei_kikan','sakuhin_detail'])

In [369]:
meguro_info = [
    date_extract(dateormovie) if dateormovie.attrs['id'] == 'jyouei_kikan' 
    else movie_extract(dateormovie)
    for dateormovie in datesandmovies]

In [376]:
def date_extract(soup):
    datespans = soup.find_all(class_='date')
    dates = [date.text.replace('.','/') for date in datespans]
    return dates

def movie_extract(soup):
    pattern = r"タイトル☆-->.*<!--☆ここまで"
    engtitle = soup.find('span', class_='text_small').text
    jptitle = re.findall(pattern, str(soup))[0].replace('\u3000', ' ')
    movie_dict = {'engtitle':engtitle, 'jptitle':jptitle}
    return(movie_dict)

In [382]:
def meguro_info(soup):
    dateormovie = soup.find_all(id=['jyouei_kikan','sakuhin_detail'])
    meguro_info = [
    date_extract(dateormovie) if dateormovie.attrs['id'] == 'jyouei_kikan' 
    else movie_extract(dateormovie)
    for dateormovie in dateormovie]
    return meguro_info

In [385]:
abc = meguro_info(soup)

In [387]:
abc

[['4/30', '5/6'],
 {'engtitle': 'GHOSTBUSTERS: AFTERLIFE', 'jptitle': 'ゴーストバスターズ アフターライフ'},
 {'engtitle': 'GHOSTBUSTERS', 'jptitle': 'ゴーストバスターズ'},
 ['5/7', '5/13'],
 {'engtitle': 'SPIDER-MAN: NO WAY HOME', 'jptitle': 'スパイダーマン：ノー・ウェイ・ホーム'},
 {'engtitle': 'UNCHARTED', 'jptitle': 'アンチャーテッド'},
 {'engtitle': 'SPIDER-MAN: INTO THE SPIDER-VERSE',
  'jptitle': 'スパイダーマン：スパイダーバース'},
 ['5/14', '5/15'],
 {'engtitle': '', 'jptitle': 'ハッピーアワー'},
 ['5/14', '5/20'],
 {'engtitle': 'EL PLANETA', 'jptitle': 'エル プラネタ'},
 {'engtitle': 'LAS NINAS / SCHOOLGIRLS', 'jptitle': 'スクールガールズ'},
 ['5/21', '5/27'],
 {'engtitle': 'The Blues Brothers', 'jptitle': 'ブルース・ブラザース'},
 {'engtitle': '（あるいは、革命がテレビ放映されなかった時）\nSUMMER OF SOUL (OR, WHEN THE REVOLUTION COULD NOT BE TELEVISED)\n',
  'jptitle': 'サマー・オブ・ソウル<span class="text_small">（あるいは、革命がテレビ放映されなかった時）'},
 {'engtitle': 'BELUSHI', 'jptitle': 'ＢＥＬＵＳＨＩ ベルーシ'},
 ['5/28', '6/3'],
 {'engtitle': '', 'jptitle': '新幹線大爆破'},
 {'engtitle': '', 'jptitle': '家族ゲーム'},
 {'engtitle': ''